<h1 align = "center",style ="font-size":40px> Hull-White Model for Bond Derivative Pricing</h1>

<h2>Modelling Interest Rates with a Trinomial Tree</h2>
Proposed in 1990, the Hull-White model<a href="#ref1"><sup>[1]</sup></a> assumes that the instantaneous interest rate $r$ is governed by a stochastic differential equation

<a id ="SDE"></a>
$$
\begin{equation} \tag{1}
dr_t = [\theta(t) - a r_t]dt + \sigma dW_t
\end{equation}
$$
where $a$ and $\sigma$ are constants, and $dW_t$ is a standard Wiener process of variance $dt$. This notebook describes how to price a bond using equation <a href="#SDE">(1)</a> via trinomial trees. The tree will be constructed in two stages, closely following the exposition in Hull's textbook <a href="#ref2"><sup>[2]</sup></a>.
<h3>Stage I</h3>
We first assume that the interest rate over a short time $\Delta t$ follows the same process as $r$: 

$$
\begin{equation*} 
dR_t = [\theta(t) - a R_t]dt + \sigma dW_t
\end{equation*}
$$
Next, we assume that $R$ can move in one of three directions at any point: up($R \to R + \Delta R $), down($R \to R - \Delta R$) and middle($R \to R$). The evolution of $R$ can therefore be described by a trinomial tree, with an additional complication that the branch structure varies based on $j$(see figure below). 

<center>
  <img src="https://github.com/praveer-kg/Finance_Public/blob/main/Option%20Pricing/gifs/Branches.png?raw=true" alt="Caption for the image">
  <figcaption><b>Fig.1</b> Branching methods.</figcaption>
</center>

Hull and White found that in order to maintain positive probabilites throughout, it was necessary to switch from the standard branching scheme (a) to the branching scheme (b) for highly positive values of $j$ and from (a) to (c) for highly negative values of $j$. The limits they provided are 
$$j_{\text{max}} = [0.184/(a \Delta t)], \quad j_{\text{min}} = -j_{\text{max}}$$
where $[.]$ is the step function. 

Furthermore, they first construct a tree for a variable $R^*$(rather than $R$) that is initially zero and follows the process

<a id ="SDE-R"></a>
$$
\begin{equation}\tag{2}
dR_t^* = - a R_t^* dt + \sigma dW_t
\end{equation}
$$
Once $R^*$ is modelled by a trinomial tree, we will offset it by $\alpha(t)$ in order to obtain $R(t)$ in the second stage. Equation <a href="#SDE-R">(2)</a>  tells us that the variable $dR_t^*$ is randomly distributed, with a mean of $-a R_t^*dt$ and a variance of $\sigma^2 \Delta t$:
$$
\begin{align}
\mathbb{E}[R^*(t + \Delta t)- R^*(t)] &= - aR(t)^*dt\\
\text{Var}[-a R_t^*dt]&=\sigma^2 \Delta t
\end{align}
$$
Conservation of probability gives us a third equation for $p_u, p_d$ and  $p_m$ at the $j^{th}$ node, forming a system of equations that can be algebraically solved:
$$
\begin{align*}
p_u \Delta R - p_d \Delta R &= - a j \Delta R \Delta t\\
p_u \Delta R^2 + p_d \Delta R^2 &= \sigma^2 \Delta t + a^2 j^2 \Delta R^2 \Delta t^2\\
p_u + p_m + p_d &= 1
\end{align*}
$$
The spacing between adjacent nodes is $\Delta R \equiv \sigma \sqrt{3 \Delta t}$, which yields
$$
\begin{align*}
p_u &=  \frac{1}{6} + \frac{1}{2} (a^2 j^2 \Delta t^2 - a j \Delta t)\\
p_m &= \frac{2}{3} - a^2 j^2 \Delta t^2\\
p_d &=  \frac{1}{6} + \frac{1}{2} (a^2 j^2 \Delta t^2 + a j \Delta t)
\end{align*}
$$
Similarly, for the branching shown in (b), we have:
$$
\begin{align*}
p_u &=  \frac{1}{6} + \frac{1}{2} (a^2 j^2 \Delta t^2 + a j \Delta t)\\
p_m &= -\frac{1}{3} - a^2 j^2 \Delta t^2 - 2 a j \Delta t\\
p_d &=  \frac{7}{6} + \frac{1}{2} (a^2 j^2 \Delta t^2 + 3a j \Delta t)
\end{align*}
$$

Finally, for the last branching scheme depicted in (c),
$$
\begin{align*}
p_u &=  \frac{1}{6} + \frac{1}{2} (a^2 j^2 \Delta t^2 - 3a j \Delta t)\\
p_m &= -\frac{1}{3} - a^2 j^2 \Delta t^2 + 2 a j \Delta t\\
p_d &=  \frac{7}{6} + \frac{1}{2} (a^2 j^2 \Delta t^2 - a j \Delta t)
\end{align*}
$$
As a demonstration and a sanity check, I will re-produce the tree from the book with $\sigma = 0.01, a = 0.1$ and $\Delta t=1$ y. These values result in $\Delta R = 0.0173$ and $j_{\text{max}} = 2$. The cell below defines these parameters and imports the required modules:


In [1]:
import numpy as np

a = 0.1
N = 3
sigma = 0.01
Δt = 1
ΔR = 0.01 * np.sqrt(3*Δt)
j_max = int(np.ceil(0.184 / (a * Δt)))

We now define a function that outputs the probabilities $p_u, p_m$ and $p_d$ based on $a, j$ and $\Delta t$.<a id ="probs"></a>
 

In [2]:
def probs(a,j,Δt):
    ajΔt = a*j*Δt
    j_max = int(np.ceil(0.184 / (a * Δt)))

    if abs(j) <= j_max:
        p_u = (1/6) + 0.5 * (ajΔt**2 - ajΔt)
        p_m = (2/3) - ajΔt**2
        p_d = (1/6) + 0.5 * (ajΔt**2 + ajΔt)

        return p_u, p_m, p_d
    elif j < 0:
        p_u = (1/6) + 0.5 * (ajΔt**2 + ajΔt)
        p_m = -(1/3) - ajΔt**2 - 2 * ajΔt
        p_d = (7/6) + 0.5 * (ajΔt**2 + 3 * ajΔt)

        return p_u, p_m, p_d
    else:
        p_u = (7/6) + 0.5 * (ajΔt**2 - 3*ajΔt)
        p_m = -(1/3) - ajΔt**2 + 2 * ajΔt
        p_d = (1/6) + 0.5 * (ajΔt**2 - ajΔt)
        return p_u, p_m, p_d

With that, we are ready to construct our tree, which will be represented by a dictionary with node coordinated $(i,j)$ as keys and the respective interest rates $R^*_{i,j}$ as the values. Note that $i$ represents the time $i \Delta t$ and $j$ represents the node position at that time. 

In [3]:
R_star = {}
for t in range(N + 1):
    for j in range(-t, t + 1):
        if abs(j) <= j_max:
            value = j
        elif j > 0:
            value = j - 1
        else:
            value = j + 1
            
        R_star[(t, j)] = value * ΔR

The figure below depicts the tree ```R_star```(see <a href = "https://github.com/praveer-kg/Finance_Public/blob/main/Option%20Pricing/gifs/Graph2.png">here</a> for another version with the transition probabilities printed on the edges).

<center>
  <img src="https://github.com/praveer-kg/Finance_Public/blob/main/Option%20Pricing/gifs/Graph.png?raw=true" alt="Caption for the image">
  <figcaption><b>Fig.2</b> Trinomial tree with modified branching at the penultimate time step.</figcaption>
</center>

<h3>Stage II</h3>
The next step is to calculate the offset $\alpha(t)$ such that $R(t) \equiv R^*(t) + \alpha(t)$ matches the initial term structure of interest rates. Hull provides the following table of interest rates for the tree we have just constructed:

| Maturity(y) | Rate(%) |
|-------------------|-------|
| 0.5               | 3.430 |
| 1.0               | 3.824 |
| 1.5               | 4.183 |
| 2.0               | 4.512 |
| 2.5               | 4.812 |
| 3.0               | 5.086 |
<b>Table 1</b>: Sample yields for example tree

The initial value $\alpha_0$ is straightforward since we've chosen $R^*$ to be initially zero: it is simply the rate corresponding to the first $\Delta t$ period(i.e, $\alpha_0 = 3.824 \%$). However, for later times, we will need to first calculate $Q_{i,j}$, which is the present value of a security that pays off $ \$1$ if node $(i,j)$ is reached, and zero otherwise. There is a simple relationship between the bond price $p$ at node $(m+1,j)$, the $\alpha$ at time $m\Delta t$ and $Q_{m,j}$ at the node $(m,j)$ :
$$
\begin{align}
P_{m+1} = \sum_{k=-n_m}^{n_m} Q(m,k) \exp[-(\alpha_m + k \Delta R)\Delta t]\\
\end{align}
$$
where $n_m$ is the number of positive/negative nodes on either side of the zeroth node at time $m$. We can use the available interest rates to calculate the market price:
$$
P^{\text{market}}_{m+1} = \exp[-R_{m+1} T_{m+1}]
$$
where $T_{m+1} = (m+1) \Delta t$. Matching this price to the $P_{m+1}$ obtained from the tree allows us to calculate $\alpha_m$:
$$
\alpha_m = -\frac{1}{\Delta t} \ln \left( \frac{P^{\text{market}}_{m+1}}{\sum_{j} Q_{m,k}. e^{-k \Delta R \Delta t}} \right)
$$
Once $\alpha_m$ has been determined from $Q_{m,j}$, the next step is to calculate $Q_{m+1,j}$ as follows:
$$
Q_{m+1,j} = \sum_k Q_{m,k} p(k,j) \exp[-(\alpha_m + k \Delta R) \Delta t ]
$$

where $p(k,j)$ is the probability of going from node $(m,k)$ to node $(m+1,j)$. 
To summarize the procedure:
 * Calculate $Q_{m,j}$ from $\alpha_{m-1}$ 
 * Calculate the market price $P^{\text{market}}_{m+1}$ using the provided table of interest rates
 * Calculate $\alpha_m$ from $P^{\text{market}}_{m+1}$ and $Q_{m,j}$
 * Calculate $Q_{m+1,j}$ from $\alpha_m$

That is what we code below by defining arrays for interest rate values, their corresponding maturity times, and the resulting market prices

In [4]:
t_array = np.array([0,1,2,3,4])
zero_rates = np.array([0,0.03824,0.04512,0.050860, 0.05350])
P_market = np.array([np.exp(-r * T) for r, T in zip(t_array,zero_rates)])

Note that the first value($t = 0$) is just a placeholder that will not be used in our calculations. The final value, corresponding to $T+\Delta t$ is required to calculate the $\alpha$ and $Q$ corresponding to $t=T$, which here is three years. I have added an extra interest rate at $t = 4$, so that we can calculate up to $m=3$.

We will now create two  dictionaries for $\alpha$ and $Q$ to store their values with time stamps and position on the tree:

In [5]:
Q = {0: {0: 1.0}}  #Q[T_i][j] = value at node(i,j)
α = {0: zero_rates[1]}  #α0

Finally, we will implement the algorithm described above:

In [6]:
for m in range(N):
    #calculate Q[m+1]
    Q[m + 1] = {}
    for j in Q[m]:
        pu, pm, pd = probs(a,j,Δt)
        r_mj = α[m] + j * ΔR
        disc = np.exp(-r_mj * Δt)
        for dj, prob in zip([1, 0, -1], [pu, pm, pd]):
            k = j + dj
            Q[m + 1][k] = Q[m + 1].get(k, 0.0) + Q[m][j] * prob * disc    #insert Q(i,j) if it exists, zero otherwise
    #compute α_{m+1}
    discount_sum = sum(Q[m + 1][k] * np.exp(-k * ΔR * Δt) for k in Q[m + 1])
    α[m + 1] = -np.log(P_market[m + 2] / discount_sum) / Δt
print("α(t) = ",α)
print("Q = ",Q)

α(t) =  {0: 0.03824, 1: 0.052049999999991534, 2: 0.06252049999698797, 3: 0.06178720495229365}
Q =  {0: {0: 1.0}, 1: {1: 0.16041365291821671, 0: 0.6416546116728669, -1: 0.16041365291821671}, 2: {2: 0.018208983798748666, 1: 0.19979708973690788, 0: 0.4735937652476668, -1: 0.2032612151759592, -2: 0.018850814146593193}, 3: {3: 0.0014319936509400773, 2: 0.032797692051419476, 1: 0.20001730682867416, 0: 0.3805488724411025, -1: 0.20697994879135442, -2: 0.03512557968893043, -3: 0.0015888185397723504}}


The values reflect those given in the source<a href="#ref2"><sup>[2]</sup></a>, validating our implementation.

<h2>Pricing the Bond and its Derivative</h2>
Under the Hull-White model, the price at $t$ of a coupon-free bond maturing at $S$, is given by:

<a id ="Price"></a>
$$
\begin{equation}\tag{3}
P(t,T) = \hat{A}(t,T) e^{- \hat{B}(t,T)R}
\end{equation}
$$

where 
$$
\hat{B}(t,T) = \frac{B(t,T)}{B(t,t+\Delta t)} \Delta t,
$$
$$
\ln\hat{A} = \ln\left(\frac{P(0,T)}{P(0,t)} \right) - \frac{B(t,T)}{B(t,t+Δt)} \ln\left(\frac{P(0,t + \Delta t)}{P(0,t)} \right) - \frac{\sigma^2}{4a}(1-e^{-2at})B(t,T)[B(t,T) - B(t,t+Δt)]
$$
and 
$$
B(t,T) = \frac{1 - e^{-a(T-t)}}{a}
$$

A European call option on this bond, with a strike price of $K$ and time to expiry of $T$, is valued at
\begin{align*}
V_\text{Call} &=  L \cdot P(0,S)  N(h) - K \cdot P(0,T) N(h - \sigma_p) 
\end{align*}
where
\begin{align*}
\sigma_p &= \sigma \left [\frac{1 - e^{-a (S - T))}}{a} \right] \sqrt{\frac{1 - e^{-2  a  T}}{2 a}} \\
h &= \frac{1}{\sigma_p} \ln\left(\frac{L  P(0,S)}{K  P(0,T)}\right) + \frac{\sigma_p}{2}
\end{align*}

and $L$ is the face value of the bond.
The put option is similarly priced at
\begin{align*}
V_\text{Put} &=   K \cdot P(0,T) N(-h + \sigma_p) - L  \cdot P(0,S) N(-h)
\end{align*}
To calculate option prices with a trinomial tree for interest rates, we will first use the analytical expression <a href="#Price">(3)</a> to calculate the bond price $P_{i,j}$ at a node $(i,j)$ using the interest rate $\alpha_i + R^*_{i,j}$. The corresponding payoff will be

$$\text{payoff} =  \begin{cases} 
& \text{max}\{(P_{i,j}-K),0 \} \ \text{for calls}\\
 & \text{max}\{(K-P_{i,j}),0 \} \ \text{for puts}
       \end{cases}
$$

The option price is obtained simply by discounting the payoffs with a factor of $e^{- R_{i,j} \Delta t}$. We will illustrate this using a second example provided by Hull<a href="#ref2"><sup>[2]</sup></a>, where he calculates the price of a 3-year European put option on a zero-coupon bond that matures in nine years. We will set the face value of the bond at $L=100$, the strike price of the put option at $K=63$, $a = 0.1$, $\sigma = 0.01$ and $N=200$. The cell below defines these parameters and imports the relevant modules

In [7]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.stats import norm

a = 0.1
σ = 0.01
N = 200        #num. of time steps in tree
S = 9
L = 100
K = 63
T_option = 3

Δt = T_option/N
t_array = np.linspace(0,T_option+Δt,N+2)
ΔR = 0.01 * np.sqrt(3*Δt)
j_max = int(np.ceil(0.184 / (a * Δt)))

The table below lists zero rates for maturities ranging from three days to ten years. We are only interested in values up to $t = 3$ y(=$1096$ d) but we will use the entire set to interpolate the values in between the available data points, so that we have accurate enough samples for whatever $N$ we choose.

| Maturity(days) | Rate (%) |
|---|---|
| 3 | 5.01772 |
| 31 | 4.98284 |
| 62 | 4.97234 |
| 94 | 4.96157 |
| 185 | 4.99058 |
| 367 | 5.09389 |
| 731 | 5.79733 |
| 1,096 | 6.30595 |
| 1,461 | 6.73464 |
| 1,826 | 6.94816 |
| 2,194 | 7.08807 |
| 2,558 | 7.27527 |
| 2,922 | 7.30852 |
| 3,287 | 7.39790 |
| 3,653 | 7.49015 |
<b>Table 2</b>: Yield data for example #2

In [8]:
maturity_d = np.array([3, 31, 62, 94, 185, 367, 731, 1096, 1461, 1826, 2194, 2558, 2922, 3287, 3653])
maturities_y = maturity_d / 365.0

rates = np.array([5.01772, 4.98284, 4.97234, 4.96157, 4.99058, 5.09389,5.79733, 6.30595, 6.73464, 6.94816, 7.08807, 7.27527,7.30852, 7.39790, 7.49015]) / 100.0
R_interp = interp1d(maturities_y, rates, kind='linear', fill_value='extrapolate')
zero_rates =np.array([R_interp(t) for t in t_array]) 
P_market = np.array([np.exp(-R * t) for t, R in zip(t_array,zero_rates)])

We will now define functions to calculate bond prices as well as the analytical put price thereof. The probability function defined in <a href="#probs">cell #2</a> still applies.

In [9]:
def P0(t):
    return np.exp(-R_interp(t) * t)

def B(t,S, a):
    return (1 - np.exp(-a * (S - t))) / a

def Bhat(t, S, a, Δt):
    return Δt * B(t, S, a) / B(t, t + Δt, a)

def Ahat(t, S, a, σ, Δt):
    BtT = B(t, S, a)
    Bttdt = B(t, t + Δt, a)

    term1 = np.log(P0(S) / P0(t))
    term2 = (BtT / Bttdt) * np.log(P0(t + Δt) / P0(t))
    term3 = (σ**2 / (4 * a)) * (1 - np.exp(-2 * a * t)) * BtT * (BtT - Bttdt)

    lnAh = term1 - term2 - term3
    return np.exp(lnAh)

def BP(t, S, r_t, a, σ, Δt):
    AtT = Ahat(t, S, a, σ, Δt)
    BtT = Bhat(t, S, a, Δt)
    return AtT * np.exp(-BtT * r_t)

def Sigma_p(a, σ, T, S):
    term1 = (1 - np.exp(-a * (S - T))) / a
    term2 = np.sqrt((1 - np.exp(-2 * a * T)) / (2 * a))
    return σ * term1 * term2

def Put_Ana(K, L, T, S, a, σ):
    P0T = P0(T)
    P0S = P0(S)
    σ_p = Sigma_p(a, σ, T, S)
    h = (1 / σ_p) * np.log((L * P0S) / (K * P0T)) + σ_p/2
    return K * P0T * norm.cdf(-h + σ_p) - L * P0S * norm.cdf(-h)

As a sanity check, it is a good exercise to verify that ```P0(S)``` and ```P(0, S, zero_rates[1], a, σ, Δt)``` are equal(remember that the first entry in ```t_array```&mdash;and so in ```zero_rates``` &mdash; is a dummy variable). Next, we repeat the tree building process done above.

In [10]:
R_star = {}
for t in range(N):
    for j in range(-t, t + 1):
        if abs(j) <= j_max:
            value = j
        elif j > 0:
            value = j - 1
        else:
            value = j + 1
            
        R_star[(t, j)] = value * ΔR

α = {0: R_interp(Δt)}  #initialize α
Q = {0: {0: 1.0}}
for m in range(N):
    Q[m + 1] = {}
    for j in Q[m]:
        pu, pm, pd = probs(a,j,Δt)
        r_mj = α[m] + R_star[(m, j)]
        disc = np.exp(-r_mj * Δt)
        for dj, prob in zip([1, 0, -1], [pu, pm, pd]):
            k = j + dj
            Q[m + 1][k] = Q[m + 1].get(k, 0.0) + Q[m][j] * prob * disc

    #compute α_{m+1}
    discount_sum = sum(Q[m + 1][k] * np.exp(-k * ΔR * Δt) for k in Q[m + 1])
    α[m + 1] = -np.log(P_market[m + 2] / discount_sum) / Δt

To price the bond and its option across all nodes, we will use another dictionary $V$ that stores the option prices(in this case the European put price).

In [11]:
V = {}
#calculate option price at terminal node
for j in range(-N, N + 1):
    r_Tj = α[N] + j * ΔR
    bond_price = L*BP(T_option, S, r_Tj, a, σ, Δt)
    V[(N, j)] = max(K - bond_price, 0)

#calculate option price via backward induction
for i in reversed(range(N)):
    for j in range(-i, i + 1):
        pu, pm, pd = probs(a,j,Δt)
        v_up = V.get((i + 1, j + 1), 0.0) #get V(i,j) if it exists, zero otherwise
        v_mid = V.get((i + 1, j), 0.0)
        v_down = V.get((i + 1, j - 1), 0.0)
        R_ij = α[i] + j * ΔR
        expected = pu * v_up + pm * v_mid + pd * v_down
        V[(i, j)] = np.exp(-R_ij * Δt) * expected

print(f"Tree put price(N = {N}) =  {V[(0, 0)]:.4f}")
put_price = Put_Ana(K, L, T_option, S, a, σ)
print(f"Analytical put price: {put_price:.4f}")

Tree put price(N = 200) =  1.8097
Analytical put price: 1.8093


The put price from the tree aligns pretty well with the analytical result, which in turn is in perfect agreement with the value quoted by Hull<a href="#ref2"><sup>[2]</sup></a>, validating our implementation of the algorithm. The next step will be to calibrate the model with market data, which I will leave for another notebook.

## References

<a id="ref1"></a>[1]  John Hull, Alan White, "<a href="https://academic.oup.com/rfs/article-abstract/3/4/573/1585938">*Pricing Interest-Rate-Derivative Securities*</a>" The Review of Financial Studies, Volume 3, Issue 4, October 1990, Pages 573–592

<a id="ref2"></a>[2]  Hull, J. C. (2011). "<a href="https://www.google.com/books/edition/_/FeUuAAAAQBAJ?sa=X&ved=2ahUKEwiK86_Rz8aMAxWGF1kFHerLC_0Qre8FegQIKxBE">*Options, Futures, and Other Derivatives.*</a>" (n.p.): Pearson Education. 